In [ ]:
#Importing required libraries:
import os
import requests
import pandas as pd
import pytz  # For timezone handling
from datetime import datetime, timedelta
import warnings
from urllib3.exceptions import InsecureRequestWarning
import json
from google.transit import gtfs_realtime_pb2

In [ ]:
# Read the routes.txt file
file_path = "/workspaces/DRT_App/Assets/Data/Routes.txt" 
cr_df = pd.read_csv(file_path)
cr_df.head(20)

In [ ]:
route_name = ''
route_id = ''

In [ ]:
# Read the Towards.txt file
file_path = "../data/Towards.txt" 
sd_df = pd.read_csv(file_path)
sd_df.head(20)

In [42]:
display = sd_df[(sd_df["route_id"] == route_id)]

display

,route_id,trip_id,trip_headsign,direction_id
24,302,3077__201011_Timetable_-_2025-02,B - Anderson,0
25,302,1576__201023_Timetable_-_2025-02,North Campus,0
26,302,1581__261026_Timetable_-_2025-02,Whitby Station,1


In [43]:
towards = ''
trip_headsign = ''

In [41]:
GTFS_URL = "https://drtonline.durhamregiontransit.com/gtfsrealtime/VehiclePositions"

# Fetch the GTFS-Realtime feed
response = requests.get(GTFS_URL)
if response.status_code != 200:
    print(f"❌ Failed to fetch data. HTTP Status: {response.status_code}")
    exit()

# Parse GTFS-Realtime feed
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

# Convert to DataFrame
vehicles = []
for entity in feed.entity:
    if entity.HasField("vehicle"):
        vehicle = entity.vehicle
        vehicles.append({
            "id": vehicle.vehicle.id,
            "route_id": vehicle.trip.route_id,
            "latitude": vehicle.position.latitude,
            "longitude": vehicle.position.longitude,
            "speed": vehicle.position.speed if vehicle.position.HasField("speed") else None,
            "timestamp": vehicle.timestamp
        })

df = pd.DataFrame(vehicles)
df.head()  # Show first few rows

,id,route_id,latitude,longitude,speed,timestamp
0,6109,N1,43.876751,-78.956253,None,1742715195
1,7120,N1,43.812866,-79.123650,None,1742715215
2,7124,N1,43.961102,-78.838501,None,1742715243
3,8573,N2,43.897335,-78.865898,None,1742715184
4,8578,N2,43.955383,-78.899651,None,1742715230


In [44]:
# Function to process and filter Vehicles data
def process_vehicles_data(vehicles, concat):
    filtered_vehicles = []

    for vehicle in vehicles:
        # Extract trip details
        trip_id = vehicle.get("trip_id")
        route_id = vehicle.get("route_id")
        direction_id = vehicle.get("direction_id")
        start_time = vehicle.get("start_time")
        start_date = vehicle.get("start_date")
        schedule_relationship = vehicle.get("schedule_relationship")
        vehicle_id = vehicle.get("vehicle_id")
        latitude = vehicle.get("latitude")
        longitude = vehicle.get("longitude")
        
        # Skip if not scheduled
        if schedule_relationship != "SCHEDULED":
            continue
            
        # Combine route_id and direction_id
        combined_id = str(route_id) + str(direction_id)
        
        # Filter by the provided 'concat' value
        if combined_id == concat:
            filtered_vehicles.append({
                "trip_id": trip_id,
                "route_id": route_id,
                "direction_id": direction_id,
                "vehicle_id": vehicle_id,
                "start_time": start_time,
                "start_date": start_date,
                "latitude": latitude,
                "longitude": longitude,
                "timestamp": vehicle.get("timestamp")
            })
                
    return pd.DataFrame(filtered_vehicles)

In [ ]:
concat = ''

In [45]:
# Process and filter the data
if vehicles:
    filtered_vehicles = process_vehicles_data(vehicles, concat)

    # Check if the timestamp column exists before sorting
    if 'timestamp' in filtered_vehicles.columns:
        filtered_vehicles = filtered_vehicles.sort_values(by="timestamp", ascending=True).reset_index(drop=True)
    else:
        print("⚠️ 'timestamp' column is missing, unable to sort.")
    
    if filtered_vehicles.empty:
        print("⚠️ No vehicles found for the specified 'concat' filter.")
    else:
        print("✅ Filtered vehicles data:")
        print(filtered_vehicles)

⚠️ 'timestamp' column is missing, unable to sort.
⚠️ No vehicles found for the specified 'concat' filter.


In [46]:
print("Buses operating route {} towards {}".format(route_name, trip_headsign))
filtered_vehicles

Buses operating route 302. Route 302 towards 


""
